In [ ]:
import http.client
import json
import time
import csv
import pandas as pd

In [1]:
conn = http.client.HTTPSConnection("sephora14.p.rapidapi.com")

# headers with your API key
headers = {
    'x-rapidapi-key': "ec7e79be26msh4f6ce6532db872cp199109jsn37042832960c",
    'x-rapidapi-host': "sephora14.p.rapidapi.com"
}

output_file = 'all_makeup_product_info.json'

all_products_details = []
page = 1
more_products = True

while more_products:
 
    endpoint = f"/searchByCategory?categoryID=makeup-cosmetics&page={page}&sortBy=NEW"
    
    try:
        # API request
        conn.request("GET", endpoint, headers=headers)
        
        # get the response
        res = conn.getresponse()
        data = res.read()
        decoded_data = data.decode("utf-8")
        
        # response JSON format
        products_data = json.loads(decoded_data)
        
        # checking
        if 'products' in products_data and len(products_data['products']) > 0:
            for product in products_data['products']:
                # Extract all details
                product_details = {
                    "brandName": product.get('brandName'),
                    "displayName": product['currentSku'].get('displayName'),
                    "heroImage": product['currentSku'].get('heroImage'),
                    "altImage": product['currentSku'].get('altImage'),
                    "onSaleData": product.get('onSaleData', 'NONE'),
                    "productId": product.get('productId'),
                    "rating": product.get('rating'),
                    "reviews": product.get('reviews'),
                    "targetUrl": product.get('targetUrl')
                }
                # add details to the list
                all_products_details.append(product_details)
        else:
            # no more products on this page, stop the loop
            more_products = False
        
        # move to the next page
        page += 1

        time.sleep(2)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        more_products = False

# all product details to a file
with open(output_file, 'w') as file:
    json.dump(all_products_details, file, indent=4)

conn.close()

print(f"Collected details for {len(all_products_details)} products.")


Collected details for 2405 products.


In [2]:
# Load the JSON data 
input_file = 'all_makeup_product_info.json'
output_file = 'all_makeup_product_info.csv'

# open
with open(input_file, 'r') as file:
    products_data = json.load(file)

# CSV headers
csv_headers = ["brandName", "displayName", "heroImage", "altImage", "onSaleData", "productId", "rating", "reviews", "targetUrl"]

# Open the CSV 
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.DictWriter(csvfile, fieldnames=csv_headers)

    # headers to the CSV file
    csvwriter.writeheader()

    # product's data to the CSV file
    for product in products_data:
        csvwriter.writerow(product)

print(f"Data has been written to {output_file}")


Data has been written to all_makeup_product_info.csv


In [7]:
df = pd.read_csv('all_makeup_product_info.csv')
product_ids = df['productId'].tolist()
print(product_ids)

['P508264', 'P97989778', 'P512013', 'P468207', 'P505233', 'P432298', 'P38855877', 'P500636', 'P281411', 'P509964', 'P512354', 'P512259', 'P507492', 'P512312', 'P508041', 'P481114', 'P506130', 'P500858', 'P512553', 'P506965', 'P500307', 'P504521', 'P468206', 'P503878', 'P468353', 'P504041', 'P512793', 'P512759', 'P512640', 'P506279', 'P512175', 'P512330', 'P512314', 'P471038', 'P500637', 'P510756', 'P513148', 'P511464', 'P513383', 'P511572', 'P513074', 'P511996', 'P511752', 'P468694', 'P513177', 'P483060', 'P512862', 'P511952', 'P513140', 'P509577', 'P484039', 'P481997', 'P468175', 'P511969', 'P512742', 'P512898', 'P505440', 'P512480', 'P512296', 'P500182', 'P512319', 'P503692', 'P512638', 'P509681', 'P512320', 'P513229', 'P512305', 'P507353', 'P512007', 'P512187', 'P512525', 'P512637', 'P512885', 'P512207', 'P513145', 'P468685', 'P504823', 'P505906', 'P513144', 'P513094', 'P512321', 'P500622', 'P512240', 'P512958', 'P513132', 'P460719', 'P512920', 'P512863', 'P512014', 'P512388', 'P513

In [8]:

headers = {
    'x-rapidapi-key': "ec7e79be26msh4f6ce6532db872cp199109jsn37042832960c",
    'x-rapidapi-host': "sephora14.p.rapidapi.com"
}

output_file = 'all_product_details.json'


def get_product_data(product_id):
    max_retries = 5  # Number of retries before giving up
    retries = 0
    product_data = {}  # Dictionary to hold product data

    while retries < max_retries:
        try:
            # Initialize the connection inside the loop
            conn = http.client.HTTPSConnection("sephora14.p.rapidapi.com")
            
            # GET request for the current product ID
            conn.request("GET", f"/product?productID={product_id}", headers=headers)
            res = conn.getresponse()
            data = res.read()

            # check
            if res.status == 200:
                # Store the data for the current product ID
                product_data['product_id'] = product_id
                product_data['response'] = data.decode("utf-8")
                break 
            elif res.status == 403:
                product_data['product_id'] = product_id
                product_data['error'] = f"Failed to retrieve data, Status code: {res.status}"
                break 
            elif res.status == 429:
                print(f"Rate limit exceeded for product ID: {product_id}. Retrying...")
                time.sleep(10)  
                retries += 1  # Increment the retry count
            else:
                product_data['product_id'] = product_id
                product_data['error'] = f"Failed to retrieve data, Status code: {res.status}"
                break  # Exit the loop on other errors

            # close connection after each request
            conn.close()

        except Exception as e:
            product_data['product_id'] = product_id
            product_data['error'] = f"An error occurred. Error: {str(e)}"
            retries += 1  

    if retries == max_retries:
        product_data['product_id'] = product_id
        product_data['error'] = "Max retries exceeded"

    # Write data to the file
    with open(output_file, 'a') as file:
        file.write(json.dumps(product_data) + "\n")

# Loop through each product ID
for product_id in product_ids:
    get_product_data(product_id)
    time.sleep(2) 


In [13]:

# Initialize 
all_flattened_data = []

# Open the JSON file
with open('all_product_details.json', 'r') as file:
    for line in file:
        try:
            product = json.loads(line)
            response = json.loads(product['response'])

            # Flatten the JSON
            flattened_data = {
                "Product ID": product.get("product_id", "N/A"),
                "Brand Name": response.get("brand", {}).get("displayName", "N/A"),
                "Product Name": response.get("displayName", "N/A"),
                "List Price": response.get("currentSku", {}).get("listPrice", "N/A"),
                "Size": response.get("currentSku", {}).get("size", "N/A"),
                "Rating": response.get("rating", "N/A"),
                "Number of Reviews": response.get("reviews", "N/A"),
                "Ingredient Description": response.get("currentSku", {}).get("ingredientDesc", "N/A"),
                "Full Product URL": response.get("fullSiteProductUrl", "N/A"),
            }

            # image URLs
            image_urls = response.get("currentSku", {}).get("alternateImages", [])
            for idx, image in enumerate(image_urls, start=1):
                flattened_data[f"Image {idx} URL"] = image.get("image1500", "N/A")

            # add the flattened data to the list
            all_flattened_data.append(flattened_data)

        except json.JSONDecodeError as e:
            print(f"Skipping line due to JSONDecodeError: {e}")


csv_file = "all_product_details.csv"
keys = set().union(*(d.keys() for d in all_flattened_data))

with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=keys)
    writer.writeheader()
    writer.writerows(all_flattened_data)

print(f"Data successfully written to {csv_file}")


Data successfully written to all_product_details.csv
